In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/content/performance.csv")

In [3]:
df.head()

,Unnamed: 0,ASSETME,CORR,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,excess_return,gross_leverage,long_exposure,long_value,longs_count,max_drawdown,max_leverage,net_leverage,orders,period_close,period_label,period_open,pnl,portfolio_value,positions,returns,sharpe,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
0,2017-01-03 21:00:00+00:00,216.99,-0.371008,NaN,0.000000,NaN,0.007650,NaN,NaN,0.00,10000000.00,0.0,0.0,0.0,0.000000,0.0,0.0,0,0.000000,0.000000,0.000000,[],2017-01-03 21:00:00+00:00,2017-01,2017-01-03 14:31:00+00:00,0.00,10000000.00,[],0.000000,NaN,0.0,0.0,0,NaN,10000000.00,0.0,0.0,1,[],0.0
1,2017-01-04 21:00:00+00:00,226.99,0.859145,0.000000,0.000000,0.000000,0.013645,0.019097,0.000000,0.00,10000000.00,0.0,0.0,0.0,0.000000,0.0,0.0,0,0.000000,0.000000,0.000000,[],2017-01-04 21:00:00+00:00,2017-01,2017-01-04 14:31:00+00:00,0.00,10000000.00,[],0.000000,NaN,0.0,0.0,0,NaN,10000000.00,0.0,0.0,2,[],0.0
2,2017-01-05 21:00:00+00:00,226.75,0.676593,0.000000,0.000000,0.000000,0.012840,0.070899,0.000000,0.00,10000000.00,0.0,0.0,0.0,0.000000,0.0,0.0,0,0.000000,0.000000,0.000000,"[{'id': '7455df22120e4e638269c3b9996d9821', 'd...",2017-01-05 21:00:00+00:00,2017-01,2017-01-05 14:31:00+00:00,0.00,10000000.00,[],0.000000,NaN,0.0,0.0,0,NaN,10000000.00,0.0,0.0,3,[],0.0
3,2017-01-06 21:00:00+00:00,229.01,0.394703,0.000026,-0.000003,-0.000247,0.016463,0.058148,0.000042,-62872.42,9937127.58,62840.0,62840.0,0.0,0.006284,62840.0,62840.0,1,-0.000003,0.006284,0.006284,"[{'id': '7455df22120e4e638269c3b9996d9821', 'd...",2017-01-06 21:00:00+00:00,2017-01,2017-01-06 14:31:00+00:00,-32.42,9999967.58,"[{'sid': Equity(1894 [MSFT]), 'amount': 1000, ...",-0.000003,-7.937254,0.0,0.0,0,-7.937254,10000000.00,0.0,0.0,4,"[{'amount': 1000, 'dt': Timestamp('2017-01-06 ...",0.0
4,2017-01-09 21:00:00+00:00,231.28,0.814375,0.000138,-0.000023,-0.002076,0.013108,0.072753,0.001371,0.00,9937127.58,62640.0,62640.0,0.0,0.006264,62640.0,62640.0,1,-0.000023,0.006284,0.006264,[],2017-01-09 21:00:00+00:00,2017-01,2017-01-09 14:31:00+00:00,-200.00,9999767.58,"[{'sid': Equity(1894 [MSFT]), 'amount': 1000, ...",-0.000020,-8.485758,0.0,0.0,0,-8.143788,9937127.58,62840.0,62840.0,5,[],0.0


In [4]:
df = df[['benchmark_period_return']]

In [5]:
df.head()

,benchmark_period_return
0,0.007650
1,0.013645
2,0.012840
3,0.016463
4,0.013108


In [6]:
df['lag_0'] = df.benchmark_period_return
df = df.drop(columns=['benchmark_period_return'])

In [7]:
for i in range(1, 20):
    col = "lag_{}".format(i)
    df[col] = df.lag_0.shift(-1 * i).values

In [8]:
df.head()

,lag_0,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19
0,0.007650,0.013645,0.012840,0.016463,0.013108,0.013108,0.015971,0.013421,0.015747,0.012169,0.014405,0.010647,0.014360,0.011721,0.018208,0.027021,0.025948,0.024337,0.017984,0.017895
1,0.013645,0.012840,0.016463,0.013108,0.013108,0.015971,0.013421,0.015747,0.012169,0.014405,0.010647,0.014360,0.011721,0.018208,0.027021,0.025948,0.024337,0.017984,0.017895,0.018298
2,0.012840,0.016463,0.013108,0.013108,0.015971,0.013421,0.015747,0.012169,0.014405,0.010647,0.014360,0.011721,0.018208,0.027021,0.025948,0.024337,0.017984,0.017895,0.018298,0.018968
3,0.016463,0.013108,0.013108,0.015971,0.013421,0.015747,0.012169,0.014405,0.010647,0.014360,0.011721,0.018208,0.027021,0.025948,0.024337,0.017984,0.017895,0.018298,0.018968,0.025992
4,0.013108,0.013108,0.015971,0.013421,0.015747,0.012169,0.014405,0.010647,0.014360,0.011721,0.018208,0.027021,0.025948,0.024337,0.017984,0.017895,0.018298,0.018968,0.025992,0.024158


In [9]:
df.tail()

,lag_0,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19
246,0.219956,0.218496,0.219089,0.221597,0.216992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,0.218496,0.219089,0.221597,0.216992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,0.219089,0.221597,0.216992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,0.221597,0.216992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,0.216992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df = df.dropna()

In [11]:
df.tail()

,lag_0,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19
227,0.180781,0.192760,0.192034,0.202470,0.199974,0.198522,0.194212,0.194438,0.198204,0.204738,0.208368,0.210501,0.210365,0.205419,0.215395,0.223103,0.218405,0.217767,0.220275,0.219956
228,0.192760,0.192034,0.202470,0.199974,0.198522,0.194212,0.194438,0.198204,0.204738,0.208368,0.210501,0.210365,0.205419,0.215395,0.223103,0.218405,0.217767,0.220275,0.219956,0.218496
229,0.192034,0.202470,0.199974,0.198522,0.194212,0.194438,0.198204,0.204738,0.208368,0.210501,0.210365,0.205419,0.215395,0.223103,0.218405,0.217767,0.220275,0.219956,0.218496,0.219089
230,0.202470,0.199974,0.198522,0.194212,0.194438,0.198204,0.204738,0.208368,0.210501,0.210365,0.205419,0.215395,0.223103,0.218405,0.217767,0.220275,0.219956,0.218496,0.219089,0.221597
231,0.199974,0.198522,0.194212,0.194438,0.198204,0.204738,0.208368,0.210501,0.210365,0.205419,0.215395,0.223103,0.218405,0.217767,0.220275,0.219956,0.218496,0.219089,0.221597,0.216992


In [14]:
dataset = df.to_numpy()

In [15]:
dataset.shape

(232, 20)

In [16]:
X = dataset[:, 0:15]
y = dataset[:, 15:20]

In [17]:
X.shape

(232, 15)

In [18]:
y.shape

(232, 5)

In [19]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

In [ ]:
estimator = MultiOutputRegressor(SVR(gamma='scale'))

In [ ]:
estimator.get_params().keys()

dict_keys(['estimator__C', 'estimator__cache_size', 'estimator__coef0', 'estimator__degree', 'estimator__epsilon', 'estimator__gamma', 'estimator__kernel', 'estimator__max_iter', 'estimator__shrinking', 'estimator__tol', 'estimator__verbose', 'estimator', 'n_jobs'])

In [ ]:
parameters = {
    'estimator__kernel': ('linear', 'poly', 'rbf'),
    'estimator__C': [1, 10]
}

In [ ]:
cv = TimeSeriesSplit(n_splits=5)
clf = GridSearchCV(estimator, parameters, cv=cv)

In [ ]:
clf.fit(X, y)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
       error_score='raise-deprecating',
       estimator=MultiOutputRegressor(estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
           n_jobs=None),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'estimator__kernel': ('linear', 'poly', 'rbf'), 'estimator__C': [1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
clf.cv_results_

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([0.00322628, 0.00290589, 0.00281777, 0.00276108, 0.00278578,
        0.00280643]),
 'std_fit_time': array([5.85497568e-04, 1.18873543e-04, 6.55815971e-05, 8.47839263e-05,
        5.30481285e-05, 7.74472213e-05]),
 'mean_score_time': array([0.00102067, 0.0009757 , 0.00093422, 0.00093355, 0.00092926,
        0.00093307]),
 'std_score_time': array([7.54205059e-05, 2.87405068e-05, 7.74563219e-06, 8.75953883e-06,
        5.22479507e-06, 1.28742501e-05]),
 'param_estimator__C': masked_array(data=[1, 1, 1, 10, 10, 10],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_estimator__kernel': masked_array(data=['linear', 'poly', 'rbf', 'linear', 'poly', 'rbf'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'estimator__C': 1, 'estimator__kernel': 'linear'},
  {'estimator__C': 1, 'estimator__kernel': 'poly'},
  {'estimator_

In [ ]:
clf.best_estimator_

MultiOutputRegressor(estimator=SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
           n_jobs=None)

In [ ]:
clf.best_params_

{'estimator__C': 1, 'estimator__kernel': 'linear'}

In [ ]:
clf.best_score_

-31.00434051799314

In [ ]:
import joblib

In [ ]:
joblib.dump(clf.best_estimator_, 'estimator.joblib')

['estimator.joblib']